<a href="https://colab.research.google.com/github/SaddamRafiq/Crypto-Thesis/blob/main/All_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import Required Libraries
import numpy as np
import pandas as pd
from statsmodels.tsa.ar_model import AutoReg
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from math import sqrt

# Fetch Data
data = yf.download('BTC-USD', start='2020-01-01', end='2023-01-01')
data = data[['Close']].dropna()
train_size = int(len(data) * 0.8)
train, test = data[:train_size], data[train_size:]

# Train AR Model
ar_model = AutoReg(train['Close'], lags=5).fit()
ar_predictions = ar_model.predict(start=len(train), end=len(train) + len(test) - 1, dynamic=False)

# Evaluate AR Model
def evaluate_metrics(true, predicted):
    rmse = sqrt(mean_squared_error(true, predicted))
    r2 = r2_score(true, predicted)
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    return {"RMSE": rmse, "R2": r2, "MAE": mae, "MSE": mse}

ar_metrics = evaluate_metrics(test['Close'], ar_predictions)
print("AR Model Metrics:", ar_metrics)


[*********************100%***********************]  1 of 1 completed

AR Model Metrics: {'RMSE': 12517.16103519283, 'R2': -11.80989609028057, 'MAE': 11701.177346351962, 'MSE': 156679320.38094965}



/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/deterministic.py:308: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending. The index is set will contain the position relative to the data length.
  fcast_index = self._extend_index(index, steps, forecast_index)


In [ ]:
# Import Required Libraries
from statsmodels.tsa.arima.model import ARIMA

# Train ARIMA Model
arima_model = ARIMA(train['Close'], order=(5, 1, 0)).fit()
arima_predictions = arima_model.forecast(steps=len(test))

# Evaluate ARIMA Model
arima_metrics = evaluate_metrics(test['Close'], arima_predictions)
print("ARIMA Model Metrics:", arima_metrics)


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


ARIMA Model Metrics: {'RMSE': 9632.232238298371, 'R2': -6.585562972702675, 'MAE': 9074.103087425323, 'MSE': 92779897.89251444}


In [ ]:
# Import Required Libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

# Data Preprocessing
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_train = scaler.fit_transform(train[['Close']])
scaled_test = scaler.transform(test[['Close']])

def create_sequences(data, sequence_length=60):
    X, y = [], []
    for i in range(sequence_length, len(data)):
        X.append(data[i-sequence_length:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

sequence_length = 60
X_train, y_train = create_sequences(scaled_train, sequence_length)
X_test, y_test = create_sequences(scaled_test, sequence_length)

X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Build LSTM Model
lstm_model = Sequential()
lstm_model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
lstm_model.add(LSTM(50))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mean_squared_error')

# Train LSTM Model
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

# Predict and Evaluate LSTM Model
lstm_predictions = lstm_model.predict(X_test)
lstm_predictions = scaler.inverse_transform(lstm_predictions)
aligned_test = test['Close'][sequence_length:].reset_index(drop=True)
lstm_metrics = evaluate_metrics(aligned_test, lstm_predictions.flatten())
print("LSTM Model Metrics:", lstm_metrics)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
LSTM Model Metrics: {'RMSE': 1192.4924271355549, 'R2': 0.7353130578994751, 'MAE': 866.59482421875, 'MSE': 1422038.1887756467}
